In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from sortedcontainers import SortedSet
import statistics
import operator
from statsmodels.distributions.empirical_distribution import ECDF

%matplotlib inline

In [2]:
df = pd.read_csv('/home/shane/Documents/thesis/output/cleaned/downdetector_v2.csv', parse_dates=['timestamp_dir'])
df['operational_pct'] = df['operational_pct'].astype('float')
df['partial_pct'] = df['partial_pct'].astype('float')
df['major_pct'] = df['major_pct'].astype('float')
df['maint_pct'] = df['maint_pct'].astype('float')
df

,source,service,timestamp_dir,timestamp_site,operational_count,operational_pct,partial_outage_count,partial_pct,major_outage_count,major_pct,maint_count,maint_pct
0,downdetector,airbnb,2017-10-23 06:10:01,2017-10-23T02:10:38.215732-04:00,1,1.0,0,0.0,0,0.0,0,0.0
1,downdetector,netflix,2017-10-23 06:10:01,2017-10-23T02:10:05.953817-04:00,0,0.0,1,1.0,0,0.0,0,0.0
2,downdetector,airbnb,2017-10-23 06:15:01,2017-10-23T02:15:37.447651-04:00,1,1.0,0,0.0,0,0.0,0,0.0
3,downdetector,netflix,2017-10-23 06:15:01,2017-10-23T02:15:04.442774-04:00,0,0.0,1,1.0,0,0.0,0,0.0
4,downdetector,airbnb,2017-10-23 06:20:01,2017-10-23T02:20:38.507157-04:00,1,1.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
323703,downdetector-gbr-great-britain,snapchat,2018-09-18 07:00:01,2018-09-18T08:00:11.338359+01:00,1,1.0,0,0.0,0,0.0,0,0.0
323704,downdetector-gbr-great-britain,netflix,2018-09-18 07:00:01,2018-09-18T08:00:13.774440+01:00,1,1.0,0,0.0,0,0.0,0,0.0
323705,downdetector-gbr-great-britain,amazon,2018-09-18 07:00:01,2018-09-18T08:00:11.848152+01:00,1,1.0,0,0.0,0,0.0,0,0.0
323706,downdetector-nzl-new-zealand,reddit,2018-09-18 07:00:01,2018-09-18T19:00:27.548824+12:00,1,1.0,0,0.0,0,0.0,0,0.0


In [6]:
df_grp = df.sort_values(['service','timestamp_dir']).groupby(['service','timestamp_dir'], as_index=False).agg({'operational_count': 'sum', 'operational_pct': 'mean', 'partial_outage_count': 'sum', 'partial_pct': 'mean', 'major_outage_count': 'sum', 'major_pct': 'mean', 'maint_count': 'sum', 'maint_pct': 'mean'})
df_grp.insert(0, 'source', 'downdetector')
df_grp.insert(3, 'timestamp_site', 'REMOVED')
df_grp

,source,service,timestamp_dir,timestamp_site,operational_count,operational_pct,partial_outage_count,partial_pct,major_outage_count,major_pct,maint_count,maint_pct
0,downdetector,airbnb,2017-10-23 06:10:01,REMOVED,1,1.0,0,0.0,0,0.0,0,0.0
1,downdetector,airbnb,2017-10-23 06:15:01,REMOVED,1,1.0,0,0.0,0,0.0,0,0.0
2,downdetector,airbnb,2017-10-23 06:20:01,REMOVED,1,1.0,0,0.0,0,0.0,0,0.0
3,downdetector,airbnb,2017-10-23 06:25:01,REMOVED,1,1.0,0,0.0,0,0.0,0,0.0
4,downdetector,airbnb,2017-10-23 06:30:01,REMOVED,1,1.0,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27246,downdetector,zynga,2018-09-18 03:00:01,REMOVED,5,1.0,0,0.0,0,0.0,0,0.0
27247,downdetector,zynga,2018-09-18 04:00:01,REMOVED,5,1.0,0,0.0,0,0.0,0,0.0
27248,downdetector,zynga,2018-09-18 05:00:01,REMOVED,5,1.0,0,0.0,0,0.0,0,0.0
27249,downdetector,zynga,2018-09-18 06:00:01,REMOVED,5,1.0,0,0.0,0,0.0,0,0.0


In [4]:
df_grp.to_csv(r'/home/shane/Documents/thesis/output/cleaned/downdetector_combined.csv', index=False)